#### 1. Import pandas library

In [1]:
import pandas as pd

#### 2. Import pymysql and sqlalchemy as you have learnt in the lesson of importing/exporting data 


In [2]:
import pymysql as sql
import sqlalchemy as db
import getpass

#### 3. Create a mysql engine to set the connection to the server. Check the connection details in [this link](https://relational.fit.cvut.cz/search?tableCount%5B%5D=0-10&tableCount%5B%5D=10-30&dataType%5B%5D=Numeric&databaseSize%5B%5D=KB&databaseSize%5B%5D=MB)

In [3]:
password = getpass.getpass("Insert your mysql root password: ")
engine = db.create_engine('mysql+pymysql://guest:{}@relational.fit.cvut.cz/stats'.format(password))
print("Connected to server!")

Insert your mysql root password: ········
Connected to server!


#### 4. Import the users table 

In [4]:
users = pd.read_sql_query("SELECT * FROM users",engine)

In [5]:
users.head(10)

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\n\n<p>I'm ...",0,5007,1920,-1,NaN,None
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,None
2,3,101,2010-07-19 15:34:50,Jarrod Dixon,2014-08-08 06:42:58,http://stackoverflow.com,"New York, NY","<p><a href=""http://blog.stackoverflow.com/2009...",22,19,0,3,35.0,None
3,4,101,2010-07-19 19:03:27,Emmett,2014-01-02 09:31:02,http://minesweeperonline.com,"San Francisco, CA",<p>currently at a startup in SF</p>\n\n<p>form...,11,0,0,1998,28.0,http://i.stack.imgur.com/d1oHX.jpg
4,5,6792,2010-07-19 19:03:57,Shane,2014-08-13 00:23:47,http://www.statalgo.com,"New York, NY",<p>Quantitative researcher focusing on statist...,1145,662,5,54503,35.0,None
5,6,457,2010-07-19 19:04:07,Harlan,2014-08-07 19:49:44,http://www.harlan.harris.name,District of Columbia,<ul>\n<li>PhD in CS/AI/Machine Learning/Cognit...,114,47,0,46050,41.0,None
6,7,429,2010-07-19 19:04:37,Vince,2014-09-10 21:14:12,http://bioinformatics.ucdavis.edu,"Davis, CA",I'm a recent graduate of UC Davis in Economics...,56,20,0,49514,28.0,None
7,8,6764,2010-07-19 19:04:52,csgillespie,2014-09-09 21:15:08,http://www.mas.ncl.ac.uk/~ncsg3/,"Newcastle, United Kingdom",<p>I'm a statistics lecturer at Newcastle Univ...,1089,604,25,70002,36.0,None
8,10,121,2010-07-19 19:05:40,Pierre,2014-07-28 18:15:02,http://plindenbaum.blogspot.com,France,Bioinformatician\\nVirology\\nGenetics\\nBiolo...,20,2,0,23234,44.0,None
9,11,136,2010-07-19 19:06:02,wahalulu,2014-07-23 21:28:18,http://www.linkedin.com/in/marckvaisman,"Washington, DC","<p>data_stuff &lt;- paste('data', c('scientist...",10,10,0,47893,40.0,None


#### 5. Rename Id column to userId

In [6]:
users.rename(columns={'Id':'userId'},inplace=True)

In [7]:
users.head(10)

,userId,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\n\n<p>I'm ...",0,5007,1920,-1,NaN,None
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,None
2,3,101,2010-07-19 15:34:50,Jarrod Dixon,2014-08-08 06:42:58,http://stackoverflow.com,"New York, NY","<p><a href=""http://blog.stackoverflow.com/2009...",22,19,0,3,35.0,None
3,4,101,2010-07-19 19:03:27,Emmett,2014-01-02 09:31:02,http://minesweeperonline.com,"San Francisco, CA",<p>currently at a startup in SF</p>\n\n<p>form...,11,0,0,1998,28.0,http://i.stack.imgur.com/d1oHX.jpg
4,5,6792,2010-07-19 19:03:57,Shane,2014-08-13 00:23:47,http://www.statalgo.com,"New York, NY",<p>Quantitative researcher focusing on statist...,1145,662,5,54503,35.0,None
5,6,457,2010-07-19 19:04:07,Harlan,2014-08-07 19:49:44,http://www.harlan.harris.name,District of Columbia,<ul>\n<li>PhD in CS/AI/Machine Learning/Cognit...,114,47,0,46050,41.0,None
6,7,429,2010-07-19 19:04:37,Vince,2014-09-10 21:14:12,http://bioinformatics.ucdavis.edu,"Davis, CA",I'm a recent graduate of UC Davis in Economics...,56,20,0,49514,28.0,None
7,8,6764,2010-07-19 19:04:52,csgillespie,2014-09-09 21:15:08,http://www.mas.ncl.ac.uk/~ncsg3/,"Newcastle, United Kingdom",<p>I'm a statistics lecturer at Newcastle Univ...,1089,604,25,70002,36.0,None
8,10,121,2010-07-19 19:05:40,Pierre,2014-07-28 18:15:02,http://plindenbaum.blogspot.com,France,Bioinformatician\\nVirology\\nGenetics\\nBiolo...,20,2,0,23234,44.0,None
9,11,136,2010-07-19 19:06:02,wahalulu,2014-07-23 21:28:18,http://www.linkedin.com/in/marckvaisman,"Washington, DC","<p>data_stuff &lt;- paste('data', c('scientist...",10,10,0,47893,40.0,None


#### 6. Import the posts table. 

In [8]:
posts = pd.read_sql_query("SELECT * FROM posts",engine)

In [9]:
posts.head(10)

,Id,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,OwnerUserId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaT,NaT,NaN,NaT,None,None
1,2,1,59.0,2010-07-19 19:12:57,22,8198.0,<p>In many different statistical methods there...,24.0,2012-11-12 09:21:54,What is normality?,...,7.0,1,8.0,88.0,2010-08-07 17:56:44,NaT,NaN,NaT,None,None
2,3,1,5.0,2010-07-19 19:13:28,54,3613.0,<p>What are some valuable Statistical Analysis...,18.0,2013-05-27 14:48:36,What are some valuable Statistical Analysis op...,...,19.0,4,36.0,183.0,2011-02-12 05:50:03,2010-07-19 19:13:28,NaN,NaT,None,None
3,4,1,135.0,2010-07-19 19:13:31,13,5224.0,<p>I have two groups of data. Each with a dif...,23.0,2010-09-08 03:00:19,Assessing the significance of differences in d...,...,5.0,2,2.0,NaN,NaT,NaT,NaN,NaT,None,None
4,5,2,NaN,2010-07-19 19:14:43,81,NaN,"<p>The R-project</p>\n\n<p><a href=""http://www...",23.0,2010-07-19 19:21:15,None,...,NaN,3,NaN,23.0,2010-07-19 19:21:15,2010-07-19 19:14:43,3.0,NaT,None,None
5,6,1,NaN,2010-07-19 19:14:44,152,29229.0,"<p>Last year, I read a blog post from <a href=...",5.0,2014-05-29 03:54:31,The Two Cultures: statistics vs. machine learn...,...,15.0,5,137.0,22047.0,2013-06-07 06:38:10,2010-08-09 13:05:50,NaN,NaT,None,None
6,7,1,18.0,2010-07-19 19:15:59,76,5808.0,<p>I've been working on a new method for analy...,38.0,2013-12-28 06:53:10,Locating freely available data samples,...,24.0,3,79.0,253.0,2013-09-26 21:50:36,2010-07-20 20:50:48,NaN,NaT,None,None
7,8,1,NaN,2010-07-19 19:16:21,0,288.0,"<p>Sorry, but the emptyness was a bit overwhel...",37.0,2010-10-18 07:57:31,So how many staticians *does* it take to screw...,...,1.0,2,NaN,449.0,2010-10-18 07:57:31,NaT,NaN,2010-07-19 20:19:46,None,None
8,9,2,NaN,2010-07-19 19:16:27,13,NaN,"<p><a href=""http://incanter.org/"">Incanter</a>...",50.0,2010-07-19 19:16:27,None,...,NaN,3,NaN,NaN,NaT,2010-07-19 19:16:27,3.0,NaT,None,None
9,10,1,1887.0,2010-07-19 19:17:47,23,21925.0,<p>Many studies in the social sciences use Lik...,24.0,2012-10-23 17:33:41,Under what conditions should Likert scales be ...,...,4.0,4,12.0,919.0,2011-03-30 15:31:46,NaT,NaN,NaT,None,None


#### 7. Rename Id column to postId and OwnerUserId to userId

In [10]:
posts.rename(columns={'Id':'postId','OwnerUserId':'userId'},inplace=True)
posts

,postId,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,userId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaT,NaT,NaN,NaT,None,None
1,2,1,59.0,2010-07-19 19:12:57,22,8198.0,<p>In many different statistical methods there...,24.0,2012-11-12 09:21:54,What is normality?,...,7.0,1,8.0,88.0,2010-08-07 17:56:44,NaT,NaN,NaT,None,None
2,3,1,5.0,2010-07-19 19:13:28,54,3613.0,<p>What are some valuable Statistical Analysis...,18.0,2013-05-27 14:48:36,What are some valuable Statistical Analysis op...,...,19.0,4,36.0,183.0,2011-02-12 05:50:03,2010-07-19 19:13:28,NaN,NaT,None,None
3,4,1,135.0,2010-07-19 19:13:31,13,5224.0,<p>I have two groups of data. Each with a dif...,23.0,2010-09-08 03:00:19,Assessing the significance of differences in d...,...,5.0,2,2.0,NaN,NaT,NaT,NaN,NaT,None,None
4,5,2,NaN,2010-07-19 19:14:43,81,NaN,"<p>The R-project</p>\n\n<p><a href=""http://www...",23.0,2010-07-19 19:21:15,None,...,NaN,3,NaN,23.0,2010-07-19 19:21:15,2010-07-19 19:14:43,3.0,NaT,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91971,115374,2,NaN,2014-09-13 23:45:39,2,NaN,"<p>This grew too long for a comment, but I thi...",805.0,2014-09-14 02:05:41,None,...,NaN,2,NaN,805.0,2014-09-14 02:05:41,NaT,115367.0,NaT,None,None
91972,115375,1,NaN,2014-09-13 23:46:05,0,9.0,<p>Assume a classification problem where there...,49365.0,2014-09-14 02:09:23,Detecting a consistent pattern in a dataset vi...,...,1.0,0,NaN,NaN,NaT,NaT,NaN,NaT,None,None
91973,115376,1,NaN,2014-09-14 01:27:54,1,5.0,<p>My goal is to create a formula that can giv...,55746.0,2014-09-14 01:40:55,How to project video viewcount based on histor...,...,0.0,2,NaN,7290.0,2014-09-14 01:40:55,NaT,NaN,NaT,None,None
91974,115377,2,NaN,2014-09-14 02:03:28,0,NaN,<p>As a practical answer to the real questions...,805.0,2014-09-14 02:54:13,None,...,NaN,0,NaN,805.0,2014-09-14 02:54:13,NaT,115358.0,NaT,None,None


#### 8. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userID,ViewCount,CommentCount

In [12]:
users_new = users[["userId","Reputation","Views","UpVotes","DownVotes"]]

In [13]:
posts_new = posts[["postId","Score","userId","ViewCount","CommentCount"]]

In [16]:
users_new.head(10)

,userId,Reputation,Views,UpVotes,DownVotes
0,-1,1,0,5007,1920
1,2,101,25,3,0
2,3,101,22,19,0
3,4,101,11,0,0
4,5,6792,1145,662,5
5,6,457,114,47,0
6,7,429,56,20,0
7,8,6764,1089,604,25
8,10,121,20,2,0
9,11,136,10,10,0


In [17]:
posts_new.head(10)

,postId,Score,userId,ViewCount,CommentCount
0,1,23,8.0,1278.0,1
1,2,22,24.0,8198.0,1
2,3,54,18.0,3613.0,4
3,4,13,23.0,5224.0,2
4,5,81,23.0,NaN,3
5,6,152,5.0,29229.0,5
6,7,76,38.0,5808.0,3
7,8,0,37.0,288.0,2
8,9,13,50.0,NaN,3
9,10,23,24.0,21925.0,4


#### 8. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [34]:
users_posts = pd.merge(users_new,posts_new, on = 'userId')

In [35]:
users_posts.head(10)

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,ViewCount,CommentCount
0,-1,1,0,5007,1920,2175,0,NaN,0
1,-1,1,0,5007,1920,8576,0,NaN,0
2,-1,1,0,5007,1920,8578,0,NaN,0
3,-1,1,0,5007,1920,8981,0,NaN,0
4,-1,1,0,5007,1920,8982,0,NaN,0
5,-1,1,0,5007,1920,9857,0,NaN,0
6,-1,1,0,5007,1920,9858,0,NaN,0
7,-1,1,0,5007,1920,9860,0,NaN,0
8,-1,1,0,5007,1920,10130,0,NaN,0
9,-1,1,0,5007,1920,10131,0,NaN,0


#### 9. How many missing values do you have in your merged dataframe? On which columns?

In [36]:
null_cols = users_posts.isnull().sum()

In [37]:
null_cols

userId              0
Reputation          0
Views               0
UpVotes             0
DownVotes           0
postId              0
Score               0
ViewCount       48396
CommentCount        0
dtype: int64

#### 10. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [38]:
users_posts["ViewCount"].fillna(0, inplace=True)

In [39]:
# Los voy a rellenar con el valor 0, porque entiendo que el ViewCount es información importante de las estadísticas y creo que no se debería borrar.

In [40]:
users_posts.head(10)

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,ViewCount,CommentCount
0,-1,1,0,5007,1920,2175,0,0.0,0
1,-1,1,0,5007,1920,8576,0,0.0,0
2,-1,1,0,5007,1920,8578,0,0.0,0
3,-1,1,0,5007,1920,8981,0,0.0,0
4,-1,1,0,5007,1920,8982,0,0.0,0
5,-1,1,0,5007,1920,9857,0,0.0,0
6,-1,1,0,5007,1920,9858,0,0.0,0
7,-1,1,0,5007,1920,9860,0,0.0,0
8,-1,1,0,5007,1920,10130,0,0.0,0
9,-1,1,0,5007,1920,10131,0,0.0,0


#### 11. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [41]:
users_posts.dtypes

userId            int64
Reputation        int64
Views             int64
UpVotes           int64
DownVotes         int64
postId            int64
Score             int64
ViewCount       float64
CommentCount      int64
dtype: object

In [49]:
# I am going to change ViewCount to integer, to avoid future issues.

users_posts["ViewCount"] = users_posts.astype({'ViewCount':'int64'})

In [50]:
users_posts.dtypes

userId          int64
Reputation      int64
Views           int64
UpVotes         int64
DownVotes       int64
postId          int64
Score           int64
ViewCount       int64
CommentCount    int64
dtype: object

#### Bonus: Identify extreme values in your merged dataframe as you have learned in class, create a dataframe called outliers with the same columns as our data set and calculate the bounds. The values of the outliers dataframe will be the values of the merged_df that fall outside that bounds. You will need to save your outliers dataframe to a csv file on your-code folder.